In [1]:
!pip install rectools[lightfm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 5.2 MB/s eta 0:00:00
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808330 sha256=45fd4f3b585a827211da34e3b7315a7d80bc50f13100002dd2fdfccdf866e49d
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from tqdm import tqdm
from pathlib import Path
from rectools import Columns

from lightfm import LightFM

from rectools.dataset import Dataset
from rectools.metrics import Precision, Recall, MAP, MeanInvUserFreq, NDCG, Serendipity, calc_metrics, AvgRecPopularity
from rectools.models import PopularModel, RandomModel, PureSVDModel, ImplicitALSWrapperModel, LightFMWrapperModel
from rectools.model_selection import TimeRangeSplitter, cross_validate

In [3]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='Downloading the kion dataset...',
                        total=total_size_in_bytes,
                        unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

In [4]:
!unzip kion_train.zip -x '__MACOSX/*'

Archive:  kion_train.zip
   creating: data_original/
  inflating: data_original/interactions.csv  
  inflating: data_original/users.csv  
  inflating: data_original/items.csv  


In [5]:
BASE_PATH = Path("./data_original")

In [6]:
interactions = pd.read_csv(BASE_PATH / "interactions.csv")
users = pd.read_csv(BASE_PATH / "users.csv")
items = pd.read_csv(BASE_PATH / "items.csv")

In [7]:
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [8]:
users.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [10]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [11]:
# переименовываем поля в понятные для RecTools

interactions = interactions.rename(
    columns={
        'total_dur': Columns.Weight,
        'last_watch_dt': Columns.Datetime
    }
)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

In [12]:
min_date = interactions['datetime'].min()
max_date = interactions['datetime'].max()

print(f"min date in interactions: {min_date}")
print(f"max date in interactions: {max_date}")

min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


In [14]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
N_FACTORS = 4

In [16]:
models = {}

In [17]:
lightfm_losses = ('logistic', 'bpr', 'warp')

for loss in lightfm_losses:
    models[f"LightFM_{loss}_factors_{N_FACTORS}"] = LightFMWrapperModel(
        LightFM(
            no_components=N_FACTORS,
            loss=loss,
            random_state=RANDOM_STATE,
        ),
        epochs=10,
        num_threads=NUM_THREADS,
    )

In [18]:
models

{'LightFM_logistic_factors_4': <rectools.models.lightfm.LightFMWrapperModel at 0x7ae5a00ae8c0>,
 'LightFM_bpr_factors_4': <rectools.models.lightfm.LightFMWrapperModel at 0x7ae5a00af250>,
 'LightFM_warp_factors_4': <rectools.models.lightfm.LightFMWrapperModel at 0x7ae5a00aea40>}

In [25]:
# Ваша схема валидации

# Переименовываем колонки
interactions = interactions.rename(columns={'total_dur': Columns.Weight,
                                            'last_watch_dt': Columns.Datetime})

# Создаем датасет
# Измените датасет при желании
binary_interactions = interactions.copy()
binary_interactions[Columns.Weight] = 1
current_dataset = Dataset.construct(
    interactions_df=binary_interactions,
    user_features_df=None,
    item_features_df=None
)

test_models = models  # Добавьте сюда любое количество моделей, которое вы тестируете

k = 10
metrics = {
    "precision": Precision(k), # Это целевая метрика для задания, остальные можно убрать если они не интересны
    "novelty": MeanInvUserFreq(k), # Это целевая метрика для задания, остальные можно убрать если они не интересны
}

splitter = TimeRangeSplitter(  # time-based валидация
    "7D",  # скользящее окно по 7 дней
    n_splits=1,  # 1 фолд (делаем для ускорения расчётов)
    filter_cold_users=True,  # только hot юзеры в тесте
    filter_cold_items=True,  # только hot айтемы в тесте
    filter_already_seen=True  # дропаем из теста просмотры трейна
)

res = cross_validate(
    current_dataset,
    splitter,
    metrics,
    test_models,
    k=10,
    filter_viewed=True,
)
metrics_res = pd.DataFrame(res["metrics"])

# Результаты можно сохранить в csv для логирования экспериментов
metrics_res.to_csv("metric_result_final.csv", index=False)

In [26]:
metrics = pd.read_csv("/content/metric_result_final.csv")
metrics

,model,i_split,precision,novelty
0,LightFM_logistic_factors_4,0,0.032490,3.715885
1,LightFM_bpr_factors_4,0,0.019785,6.523490
2,LightFM_warp_factors_4,0,0.034850,4.097372
